# Part 3 - Summarization

For Part 3, use the same 300-word long sections as in Part 1, but instead of answering the questions about these sections, ask the ChatPI to summarize them for you, using Hugging Face summarization pipeline.

Part 3.2 -- use two different HF summarization models: use the default summarization pipeline, then use other models of choice and discuss the differences in the result.

https://huggingface.co/docs/transformers/main_classes/pipelines

https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/pipelines#transformers.SummarizationPipeline


In [ ]:
!pip3 install -r ../requirements.txt

%load_ext autoreload
%autoreload 2


In [ ]:
from collections import defaultdict

from src import utils
from src.summarization import run, run_models
import evaluate

---
---

## Experiments & Results

Use the same 300-word long sections as in Part 1, but instead of answering the questions about these sections, ask the ChatPI to summarize them for you, using Hugging Face summarization pipeline.


In [ ]:
# TODO: Try out a good selection of models and keep some interesting ones
models = [
    "sshleifer/distilbart-cnn-12-6",
    "slauw87/bart_summarisation",
    "pszemraj/pegasus-x-large-book-summary",
]


In [27]:
"""
Rouge1: unigram (1-gram) based scoring
Rouge2: bigram (2-gram) based scoring
RougeL: ROUGE-L measures the longest common subsequence between the generated summary and the reference summary.
RougeLsum: splits text using \n
"""

ctx = "../sections/resolution.0.md"
text = ""

with open(ctx, "r") as f:
    text = f.read().strip()


_, scores = run_models(text, models=models, expected_answer=text, score_type="rouge")

--------------------------------------------------------------------------------
> A map is drawn for them by an Englishman named Jonathan Small. You remember that we saw the name upon the chart in Captain Morstan's possession. He had signed it in behalf of himself and his associates,-the sign of the four, as he somewhat dramatically called it. Aided by this chart, the officers-or one of them-gets the treasure and brings it to England, leaving, we will suppose, some condition under which he received it unfulfilled. Now, then, why did not Jonathan Small get the treasure himself? The answer is obvious. The chart is dated at a time when Morstan was brought into close association with convicts. Jonathan Small did not get the treasure because he and his associates were themselves convicts and could not get away." "But that is mere speculation," said I. "It is more than that. It is the only hypothesis which covers the facts. Let us see how it fits in with the sequel. Major Sholto remains at 

Your max_length is set to 512, but your input_length is only 373. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=186)


> A map is drawn for them by an Englishman named Jonathan Small. You remember that we saw the name upon the chart in Captain Morstan's possession. He had signed it in behalf of himself and his associates,-the sign of the four, as he somewhat dramatically called it. Aided by this chart, the officers-or one of them-gets the treasure and brings it to England, leaving, we will suppose, some condition under which he received it unfulfilled. Now, then, why did not Jonathan Small get the treasure himself? The answer is obvious. The chart is dated at a time when Morstan was brought into close association with convicts. Jonathan Small did not get the treasure because he and his associates were themselves convicts and could not get away." "But that is mere speculation," said I. "It is more than that. It is the only hypothesis which covers the facts. Let us see how it fits in with the sequel. Major Sholto remains at peace for some years, happy in the possession of his treasure. Then he receives a